# Development of Algorithms for ERDDAP Dataset from 2nd Narrows HADCP Observations

Figuring out how to transform 2nd Narrows horizontal ADCP observations from
AIS CSV files into a netCDF file that is part of an ERDDAP dataset:

* Read CSV file into `pandas` dataframe
* Filter to include only 2nd Narrow HADCP rows
* Convert to `xarray.Dataset`
* Add metadata

* **BONUS:** Add observations from an hourly file to an existing netCDF file
so that we can have daily or monthly netCDF instead of hourly

In [1]:
import pandas
import xarray

In [2]:
!wc -l /opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv

229 /opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv


In [3]:
!head /opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv

"28/09/2018 05:00 - 28/09/2018 06:00  [ 28/09/2018 12:00 UTC - 28/09/2018 13:00 UTC ]"
"SoG Weather Stations"
"SoG Weather Stations"
"Name","MMSI","Time","Wind Speed","Wind Dir.","Wind Gust","Wind Gust Dir.","Air Temp.","Air Pres.","Water Level","Water Temp.","Currrent Speed","Current Dir."
"VFPA CN RAIL BRIDGE","003160171","28/09/2018 05:00","","","","","-0.1","","2.3","-0.1","2.9","85"
"VFPA CN RAIL BRIDGE","003160171","28/09/2018 05:00","","","","","-0.1","","2.3","-0.1","2.9","85"
"NPA WEATHER","993161006","28/09/2018 05:00","3","159","3","161","13.3","1014","2.5","-0.1","",""
"NPA WEATHER","993161006","28/09/2018 05:00","3","159","3","161","13.3","1014","2.5","-0.1","",""
"DUKE POINT ATON","993161010","28/09/2018 05:00","10","10","13","10","15.6","1015","2.3","12.3","0.5","88"
"DUKE POINT ATON","993161010","28/09/2018 05:01","8","11","13","10","15.4","1015","2.3","12.3","0.5","88"


In [4]:
!tail /opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv

"NPA WEATHER","993161006","28/09/2018 05:54","0","123","1","110","13.2","1014","3","-0.1","",""
"NPA WEATHER","993161006","28/09/2018 05:54","0","123","1","110","13.2","1014","3","-0.1","",""
"DUKE POINT ATON","993161010","28/09/2018 05:55","3","322","5","318","13.9","1015","2.9","11.8","0.4","77"
"VFPA CN RAIL BRIDGE","003160171","28/09/2018 05:56","","","","","-0.1","","2.8","-0.1","3","86"
"DUKE POINT ATON","993161010","28/09/2018 05:56","3","322","5","318","13.6","1015","2.9","11.8","0.4","77"
"","003160011","28/09/2018 05:57","17","312","","","14.8","","3.2","","",""
"NPA WEATHER","993161006","28/09/2018 05:57","1","169","2","160","13.4","1014","3","-0.1","",""
"VFPA CN RAIL BRIDGE","003160171","28/09/2018 05:58","","","","","-0.1","","2.9","-0.1","3","86"
"VFPA CN RAIL BRIDGE","003160171","28/09/2018 05:58","","","","","-0.1","","2.9","-0.1","3","86"



In [7]:
!wc -l /opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv

229 /opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv


In [46]:
all_df = pandas.read_csv('/opp/observations/AISDATA/20180928T120000Z-20180928T130000Z.csv', skiprows=3)
all_df

,Name,MMSI,Time,Wind Speed,Wind Dir.,Wind Gust,Wind Gust Dir.,Air Temp.,Air Pres.,Water Level,Water Temp.,Currrent Speed,Current Dir.
0,VFPA CN RAIL BRIDGE,3160171,28/09/2018 05:00,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
1,VFPA CN RAIL BRIDGE,3160171,28/09/2018 05:00,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
2,NPA WEATHER,993161006,28/09/2018 05:00,3.0,159.0,3.0,161.0,13.3,1014.0,2.5,-0.1,NaN,NaN
3,NPA WEATHER,993161006,28/09/2018 05:00,3.0,159.0,3.0,161.0,13.3,1014.0,2.5,-0.1,NaN,NaN
4,DUKE POINT ATON,993161010,28/09/2018 05:00,10.0,10.0,13.0,10.0,15.6,1015.0,2.3,12.3,0.5,88.0
5,DUKE POINT ATON,993161010,28/09/2018 05:01,8.0,11.0,13.0,10.0,15.4,1015.0,2.3,12.3,0.5,88.0
6,VFPA CN RAIL BRIDGE,3160171,28/09/2018 05:02,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
7,VFPA CN RAIL BRIDGE,3160171,28/09/2018 05:02,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
8,VFPA CN RAIL BRIDGE,3160171,28/09/2018 05:02,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
9,NaN,3160011,28/09/2018 05:02,NaN,NaN,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN


In [47]:
hadcp_df = all_df.loc[df.Name == 'VFPA CN RAIL BRIDGE'].drop_duplicates().set_index('Time')
hadcp_df

,Name,MMSI,Wind Speed,Wind Dir.,Wind Gust,Wind Gust Dir.,Air Temp.,Air Pres.,Water Level,Water Temp.,Currrent Speed,Current Dir.
Time,,,,,,,,,,,,
28/09/2018 05:00,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
28/09/2018 05:02,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
28/09/2018 05:04,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
28/09/2018 05:06,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.3,-0.1,2.9,85.0
28/09/2018 05:08,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.4,-0.1,3.0,84.0
28/09/2018 05:10,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.4,-0.1,2.9,85.0
28/09/2018 05:12,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.4,-0.1,2.9,85.0
28/09/2018 05:14,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.4,-0.1,2.9,85.0
28/09/2018 05:16,VFPA CN RAIL BRIDGE,3160171,NaN,NaN,NaN,NaN,-0.1,NaN,2.5,-0.1,2.9,85.0


In [49]:
hadcp_df \
    .dropna(axis='columns', how='all') \
    .drop(['Name', 'MMSI', 'Air Temp.', 'Water Temp.', 'Water Level'], axis='columns')

,Currrent Speed,Current Dir.
Time,,
28/09/2018 05:00,2.9,85.0
28/09/2018 05:02,2.9,85.0
28/09/2018 05:04,2.9,85.0
28/09/2018 05:06,2.9,85.0
28/09/2018 05:08,3.0,84.0
28/09/2018 05:10,2.9,85.0
28/09/2018 05:12,2.9,85.0
28/09/2018 05:14,2.9,85.0
28/09/2018 05:16,2.9,85.0
